In [ ]:
import numpy as np
import pandas as pd
import pickle

np.random.seed(1728)

#  read dataset
runLocal = True
if runLocal:
    pathRoot = 'data/'
else:
    pathRoot = '/content/drive/My Drive/colab_data/'

with open(pathRoot + 'note_sequences_voc.data', 'rb') as seq_path:
    sequences = pickle.load(seq_path)
with open(pathRoot + 'note_sequences_dict.data', 'rb') as filehandle:
    lex_to_ix = pickle.load(filehandle)
    ix_to_lex = {v: k for k, v in lex_to_ix.items()}
    vocab_size = len(lex_to_ix) + 2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Input, Sequential, Model
from tensorflow.keras.utils import Sequence

def three_gram(vocab_size):
    X = Input((1))
    
    #  Compute an embedding vector and apply it to all words in the bag X
    emb = layers.Embedding(input_dim=vocab_size, output_dim=16, mask_zero=True)
    emb_vec = emb(X)
    h = layers.Dense(vocab_size, activation='tanh')(emb_vec)
    p = layers.Softmax()(h)
    
    return Model(inputs=X, outputs=p), emb

In [ ]:
class ThreeGramGenerator(Sequence):
    def __init__(self, data, batch_size=100,
                 shuffle=True, fit=True, mini_batch_limit=np.inf):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.mini_batch_limit = mini_batch_limit
        self.indexes = []
        for si, (x, _) in enumerate(data):
            for xi in range(x.shape[0]):
                if xi > 3:
                    self.indexes.append((si, xi, -3))
                if xi > 2:
                    self.indexes.append((si, xi, -2))
                if xi > 1:
                    self.indexes.append((si, xi, -1))
                if xi + 1 < x.shape[0]:
                    self.indexes.append((si, xi, 1))
                if xi + 2 < x.shape[0]:
                    self.indexes.append((si, xi, 2))
                if xi + 3 < x.shape[0]:
                    self.indexes.append((si, xi, 3))
        np.random.shuffle(self.indexes)  # always shuffle once
        
    def __len__(self):
        return int(np.min([len(self.indexes) / self.batch_size, self.mini_batch_limit]))
    
    def __getitem__(self, index):
        index *= self.batch_size
        this_size = self.batch_size if index + self.batch_size < len(self.indexes) else len(self.indexes) - index
        X = np.zeros((this_size, 1))
        Y = np.zeros((this_size, 1, vocab_size))
        for i in range(this_size):
            X[i,:], Y[i,0,:] = self.__getsingleitem(index + i)
        return X, Y
    
    def __getsingleitem(self, index):
        (seq, stride, target) = self.indexes[index]
        (N, _) = self.data[seq]
        N = N.melody.to_numpy()
        Y = np.zeros((1,vocab_size))
        Y[0,N[stride + target]] = 1
        return N[stride], Y
    
    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        if self.shuffle:
            np.random.shuffle(self.indexes)

np.random.shuffle(sequences)  # shuffle before splitting validation set
# small ds for testing
# sequences = sequences[:10]
val_split_ix = int(0.9*len(sequences))
generator = ThreeGramGenerator(sequences[:val_split_ix])
val_gen = ThreeGramGenerator(sequences[val_split_ix:])

model, embedding = three_gram(vocab_size)

opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss="categorical_crossentropy", optimizer=opt)

model.summary()

In [ ]:
model.fit(generator, epochs=4, validation_data=val_gen)